#### Import Dependencies

In [1]:
import requests
from bs4 import BeautifulSoup
import json, os
import pandas as pd
from tqdm import tqdm
import traceback
import PyPDF2

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

#### Collect Crypto Data
- Get data for top 5k crypto by marketcap from coinmarketcap.com
- Save data to a file (Prevent usage of Api limit)

In [2]:
def get_top_5k_cryptos() -> pd.DataFrame:
    
    api_key = '1d13c476-343c-4717-b648-8a4a1b939b42'
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    
    params = {
        'start': '1',
        'limit': '5000',  # You can adjust this to get more or fewer listings
        'convert': 'USD',
        'sort': 'market_cap'
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if response.status_code == 200:
        df = pd.DataFrame(data['data'])
        return df
        print(f'Data Saved..')
    else:
        print(f"Error {response.status_code}: {data['status']['error_message']}")
        
# df = get_top_5k_cryptos()
# df.to_csv('../data/topk_crypto.csv', index=False)
# print('Collected top 5k crypto.')

#### Load the dataset

In [3]:
k = 1000
df = pd.read_csv('../data/topk_crypto.csv').head(k)
print(f'Dimensions of the sataset: {df.shape}')
display(df.sample(1))


Dimensions of the sataset: (1000, 18)


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote
547,5821,Aleph.im,ALEPH,aleph-im,53,2020-07-08T00:00:00.000Z,"['mineable', 'cosmos-ecosystem', 'ai-big-data', 'distributed-computing', 'filesharing', 'storage', 'polkadot-ecosystem', 'avalanche-ecosystem', 'solana-ecosystem', 'injective-ecosystem', 'bnb-chain', 'ftx-bankruptcy-estate', 'depin']",NaN,2.472205e+08,500000000.0,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0x27702a26126e0b3702af63ee09ac4d1a084ef628'}",548,45968548.0,1.372516e+07,NaN,2024-06-06T04:12:00.000Z,"{'USD': {'price': 0.2985771825407063, 'volume_24h': 958335.41423686, 'volume_change_24h': -10.6523, 'percent_change_1h': -0.03526277, 'percent_change_24h': -1.10923119, 'percent_change_7d': -6.25369944, 'percent_change_30d': -19.89867374, 'percent_change_60d': 52.17284833, 'percent_change_90d': -19.10340228, 'market_cap': 73814395.02515341, 'market_cap_dominance': 0.0028, 'fully_diluted_market_cap': 149288591.27, 'tvl': None, 'last_updated': '2024-06-06T04:12:00.000Z'}}"


#### Get whitepaper links for each of the topk cryptos

In [4]:
def get_whitepaper_link(slug):
    url = f'https://coinmarketcap.com/currencies/{slug}/'
    response = requests.get(url)
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    divs = soup.find_all('div', class_='sc-d1ede7e3-0 sc-7f0f401-0 gRSwoF gQoblf')
    
    for div in divs:
        if 'whitepaper' not in div.get_text().lower():
            continue
        
        # Find the whitepaper link within this div
        for a_tag in div.find_all('a', href=True):
            return a_tag['href']
    
    return None

# tqdm.pandas()
# df['whitepaper_link'] = df['slug'].progress_apply(get_whitepaper_link)
# df.to_csv(f'../data/top{k}_crypto_w_whitepaper_link.csv', index=False)


#### Loading dataset with whitepaper links

In [5]:
df = pd.read_csv('../data/top1000_crypto_w_whitepaper_link.csv')
df = df.drop(columns = ['tags', 'quote'])
df = df[df['whitepaper_link'].notna()]
print(f'Shape of dataframe: ', df.shape)
print(f'Number of whitepaper links: ', df.whitepaper_link.nunique())
display(df.head(5))

Shape of dataframe:  (423, 17)
Number of whitepaper links:  413


,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,whitepaper_link
0,1,Bitcoin,BTC,bitcoin,11101,2010-07-13T00:00:00.000Z,21000000.0,1.970808e+07,1.970808e+07,False,NaN,1,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://bitcoin.org/bitcoin.pdf
1,1027,Ethereum,ETH,ethereum,9038,2015-08-07T00:00:00.000Z,NaN,1.201491e+08,1.201491e+08,True,NaN,2,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://github.com/ethereum/wiki/wiki/White-Paper
2,825,Tether USDt,USDT,tether,87800,2015-02-25T00:00:00.000Z,NaN,1.123921e+11,1.150861e+11,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xdac17f958d2ee523a2206206994597c13d831ec7'}",3,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://tether.to/wp-content/uploads/2016/06/TetherWhitePaper.pdf
4,5426,Solana,SOL,solana,674,2020-04-10T00:00:00.000Z,NaN,4.599202e+08,5.775657e+08,True,NaN,5,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://solana.com/solana-whitepaper.pdf
5,3408,USDC,USDC,usd-coin,19594,2018-10-08T00:00:00.000Z,NaN,3.260113e+10,3.260113e+10,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}",6,NaN,NaN,NaN,2024-06-06T04:12:00.000Z,https://f.hubspotusercontent30.net/hubfs/9304636/PDF/centre-whitepaper.pdf


#### Download pdfs

In [6]:
# Function to download PDF
def download_pdf(url, file_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_name, 'wb') as file:
                file.write(response.content)
                
            if not verify_pdf(file_name):
                invalid_file_name = file_name.replace('.pdf', '_invalid.pdf')
                os.rename(file_name, invalid_file_name)
            
        else:
            print(f'Failed to download {file_name}, with url: {url}')
    except Exception as e:
        print(f'An error occurred while downloading {file_name}, {e}')
        # traceback.print_exc()
        
# Function to verify PDF structure
def verify_pdf(file_name):
    try:
        with open(file_name, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            # Check if the PDF has at least one page
            if len(reader.pages) > 0:
                return True
            else:
                return False
    except Exception as e:
        print(f'An error occurred while verifying {file_name}. Error: {e}')
        # traceback.print_exc()
        return False
        

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    url = row['whitepaper_link']
    if '.pdf' not in url:
        continue
    file_name = f"../data/whitepapers/{index + 1}_{row['slug']}.pdf"
    if not os.path.exists(file_name):
        download_pdf(url, file_name)
    
    

  1%|          | 4/423 [00:01<02:20,  2.98it/s]

An error occurred while verifying ../data/whitepapers/3_tether.pdf. Error: EOF marker not found


  7%|▋         | 31/423 [00:04<00:35, 10.92it/s]

An error occurred while verifying ../data/whitepapers/38_stellar.pdf. Error: EOF marker not found


  8%|▊         | 33/423 [00:05<00:41,  9.50it/s]

...

 93%|█████████▎| 392/423 [04:02<00:05,  5.70it/s]

Failed to download ../data/whitepapers/530_ren.pdf, with url: https://renproject.io/litepaper.pdf


 94%|█████████▍| 398/423 [05:44<03:54,  9.39s/it]

An error occurred while verifying ../data/whitepapers/543_memetoon.pdf. Error: EOF marker not found


100%|██████████| 423/423 [05:52<00:00,  1.20it/s]
